In [3]:
from PIL import Image, ImageOps
import cv2
img = cv2.imread("output_2048x2048.png")
print(img.dtype) # 'uint8'
# Open image in RGB (24-bit, uint8)
img = Image.open("canvasOfYeastCells.png").convert("RGB")

# Proportionally resize to fit largest side (2048), preserving aspect ratio
img = ImageOps.contain(img, (2048, 2048), method=Image.BICUBIC)

# Pad to square if needed (with black or other value)
padded = ImageOps.pad(img, (2048, 2048), color=(0,0,0), method=Image.BICUBIC)

# Save as PNG (lossless, 8-bit per channel)
padded.save('output_2048x2048.png')

uint8


In [4]:
from pathlib import Path
from PIL import Image

# ---- CONFIG ----------------------------------------------------------
src_path   = Path("output_2048x2048.png")   # ≡ the image you just saved: padded.save(...)
tile_size  = 512                            # 2048 // 4
prefix     = src_path.stem                  # "output_2048x2048"
out_dir    = Path(".")                      # change if you want a sub‑folder
# ----------------------------------------------------------------------

# 1) Load and sanity‑check size
img = Image.open(src_path)
if img.size != (2048, 2048):
    raise ValueError(f"Expected 2048×2048, got {img.size}")

# 2) Split into a 4x4 grid and save each tile by row x col
for row in range(4):        # 0 … 3
    for col in range(4):    # 0 … 3
        left   = col * tile_size
        upper  = row * tile_size
        right  = left + tile_size
        lower  = upper + tile_size
        tile   = img.crop((left, upper, right, lower))
        tile.save(out_dir / f"{prefix}_{row}{col}.png")

AAAAAAAAAAAAAAAAAAAAAAAAAAA




MY MATH IS WRONGITHWREGPI@ERJW"
**From**

   $$
     a(x-n)^{2}+b(y-m)^{2}+cxy \;=\; d .
   $$


**Solve for $c$**

$$
  \boxed{\,c=\dfrac{\,d-a(x-n)^{2}-b(y-m)^{2}\,}{xy}\,}
  \qquad\bigl(x\neq 0,\;y\neq 0\bigr).
$$

Showing $c$ equals the residual uniquely whenever $xy$ is non‑zero; otherwise, the $c$ term implodes as the $xy$ denominator vanishes and further information about the specific conic or application is needed.


\
Problem is that I realize nobody thinks of ellipse and shapes as the equations but it is a good start. I figured I just start off with a regular non-rotated ellipse and let user change how much it is rotated.


L
Does each locat
 __ __
|[][]
|[][]


 n1, m1
 n2, m2
LABEL TROUBLES FROM w,h to 2048,2048 canvas with offset and dim

In [ ]:
#I originally used these two functions for the ellipses as seen in the yeastSegmentationVisualPERFECTELLIPSES

def create_ellipse_mask_mathematical(w: int, h: int, coeffs: dict):
    """Pixel‑by‑pixel mask using the implicit form."""
    A = coeffs["a"]; B = coeffs["b"]; C = coeffs["c"]
    h0 = coeffs["h"]; w0 = coeffs["w"]

    scale_xy = float(max(w, h))               #put offsets in (‑1,1] range
    thresh   = 1.0 / (scale_xy * scale_xy)    #F scales by 1/scale²


    mask = np.zeros((h, w), dtype=bool)
    for y in range(h):
        dy32 = np.float32((y - w0) / scale_xy)
        for x in range(w):
            dx32 = np.float32((x - h0) / scale_xy)
            if _implicit_value(A, C, B, dx32, dy32) <= thresh:
                mask[y, x] = True
    return mask

def create_ellipse_mask_vectorized(w: int, h: int, coeffs: dict):
    """Vectorised Boolean mask via broadcasting."""
    A,  B,  C,  h0, w0 = coeffs["a"],  coeffs["b"], coeffs["c"],    coeffs["h"],    coeffs["w"]

    scale_xy = float(max(w, h))
    thresh = np.float32(1.0 / (scale_xy * scale_xy))

    y_grid, x_grid = np.ogrid[:h, :w]

    dx32 = (x_grid - h0).astype(np.float32) / scale_xy
    dy32 = (y_grid - w0).astype(np.float32) / scale_xy
    return _implicit_value(A, C, B, dx32, dy32) <= thresh

#But I realized for better data quality I should transform them to make them less perfect
from noise import pnoise2


https://geo.libretexts.org/Courses/University_of_California_Davis/GEL_056%3A_Introduction_to_Geophysics/Geophysics_is_everywhere_in_geology.../zz%3A_Back_Matter/Arctan_vs_Arctan2

'''

    Arctan takes only one input value, and therefore cannot determine which of the two quadrants the angle lies in each case. By default, it provides the angle either in Quadrant I or Quadrant IV depending on the sign of  tan(𝜃)
     . It is perhaps easier to think of this in terms of triangles, and the one value taken by arctan is really the quotient of the opposite and adjacent sides of the triangle or y/x, so  arctan(𝑦/𝑥)
     . In order to determine the correct quadrant, the sign of y and x must be known, however, the function only receives the sign of x/y, and not x and y separately and cannot distinguish quadrants.

    This can occasionally be an issue, especially if we are interested in an angle that can range from -180 to 180 degrees, such as longitude. Arctan2 is the 4-quadrant inverse tangent. It can get around the previous issue by taking both x and y in as separate arguments. Whereas the syntax for arctan is  arctan(𝑦/𝑥)
     , the syntax for arctan2 is  arctan2(𝑦,𝑥)
     . Knowing the signs of x and y separately can determine if the angle lies in any of the four quadrants. The range of arctan2 is - 𝜋
       ≤arctan2(𝑦,𝑥)≤𝜋
      or in degrees, -180 ∘≤arctan2(𝑦,𝑥)≤
      180 ∘
     .
'''

Trying to perturb the ellipses in x,y only served to frustrate me since they still needed to be vaguely centered - considing this is in 2d I chose polar but in 3d I might have had to choose between spherical and cylindrical




$$
r=\sqrt{\Delta x^2+\Delta y^2}  \quad  \text{and}    \quad  \theta = \operatorname{atan2}(\Delta y,\Delta x).
$$

'''



    def create_ellipse_mask_vectorized_perturbed(w: int, h: int, coeffs: dict,
                                             jitter: float = 0.07,
                                             noise_scale: int = 64,
                                             seed: int | None = None):
        rng = np.random.default_rng(seed)Recover pixel semi‑axes from implicit coefficients
        a = 1.0 / np.sqrt(coeffs["a"])
        b = 1.0 / np.sqrt(coeffs["b"])
        cx, cy = coeffs["h"], coeffs["w"]

        # Coordinate grids
        y, x = np.ogrid[:h, :w]
        dx, dy = x - cx, y - cy
        r_px = np.hypot(dx, dy)
        theta = np.arctan2(dy, dx)

        # Ideal ellipse radius for every pixel direction
        r_ideal = (a * b) / np.sqrt((b * np.cos(theta))**2 +
                                    (a * np.sin(theta))**2)

        # 2‑D Perlin noise field ∈ [‑1,1]
        n = np.vectorize(lambda yy, xx:
                         pnoise2(xx / noise_scale, yy / noise_scale,
                                 repeatx=w, repeaty=h, base=seed or 0))
        delta = jitter * n(*np.indices((h, w)))

        return r_px <= r_ideal * (1.0 + delta)
'''

| Code fragment | Why it is (mostly) safe | Extra guard-rail if you’re paranoid |
| --- | --- | --- |
| dx, dy = x-cx, y-cy | Pure subtraction, so the only danger is *catastrophic cancellation* when the point is near the centre. But that is exactly the region where the later atan2 / hypot branch gives perfect accuracy (difference of two exactly equal floats). | Use float64 (default) and keep cx, cy as float64 too, so cancellation causes at most 52-bit precision loss, never overflow / underflow. |
| r_px = np.hypot(dx, dy) | np.hypot rescales internally to avoid both overflow and underflow and is accurate to $\le 1$ ulp. | Nothing; it already implements the improved *Fast2Sum/FMA* variant. |
| theta = np.arctan2(dy, dx) | atan2 is well-conditioned everywhere except the origin and handles signed-zero & NaN per IEEE 754. | If you ever pass the exact centre, skip computing theta; it is unused when r_px == 0. |
| r_ideal = (a*b) / sqrt((b*cosθ)^2 + (a*sinθ)^2) | Two **divisions** and four **products**. Overflow risk when $a,b \ge 1\times10^{308}$; underflow when $a,b \le 1\times10^{-308}$. For normal cell sizes ($\le 10^4$ px) you are $\ge 300$ orders of magnitude away from either limit. | Compute with a scale factor $s=\max(a,b)$ and rewrite as:<br>num = (a/s)*(b/s)<br>den = sqrt((b/s*cosθ)^2 + (a/s*sinθ)^2)<br>r_ideal = (num/den)*s`<br>so both numerator and denominator are $\le 1$. This removes the last chance of overflow. |
| delta = jitter * noise | noise ∈ [-1, 1] by construction — Perlin keeps gradients unit-norm. | Clamp to $[-1,1]$ after generation to avoid accidental out-of-range values from custom noise. |
| r_px <= r_ideal * (1+delta) | A single subtraction in floating-point order-of-comparison. Rounding can flip the result only if the point is within $\approx 1$ ulp of the boundary. | Inflate the RHS by one machine-epsilon, e.g.<br>eps = np.finfo(r_px.dtype).eps<br>mask = r_px <= r_ideal * (1 + delta + eps)` |


---
Underflow 'np.hypot' returns denormals rather than zero, so the centre pixel stays detectable.
Overflow    impossible once you rescale the r_ideal formula (see above) because all intermediates are \lessim 1.
Cancellation only the dx,dy subtraction could trigger it and we are intentionally looking for zeros there. The subsequent 'hypot neutralises' the effect.
\
**First Order Perturbation/Condition Analysis**

$$
r(\theta; a, b) = \frac{ab}{\sqrt{(b \cos \theta)^2 + (a \sin \theta)^2}} \quad \text{(ideal radius)}
$$



and let $\varepsilon_a, \varepsilon_b, \varepsilon_\delta$ be small \textit{relative} perturbations in $a$, $b$ and in the jitter amplitude:

$$
a \rightarrow a(1 + \varepsilon_a), \quad b \rightarrow b(1 + \varepsilon_b), \quad \delta \rightarrow \delta + \varepsilon_\delta.
$$


Radius Sensitvity

Differentiation gives
$$
\frac{\partial \log r}{\partial \log a} = 1 - \frac{a^2 \sin^2 \theta}{a^2 \sin^2 \theta + b^2 \cos^2 \theta}, \quad
\frac{\partial \log r}{\partial \log b} = 1 - \frac{b^2 \cos^2 \theta}{a^2 \sin^2 \theta + b^2 \cos^2 \theta}.
$$

**Classification Stability**
The boundary test is
$$
f(\mathbf{x}) = \frac{r_{px}}{r_{\text{ideal}}(1 + \delta)} \leq 1.
$$. This is absolutely unrelated but I found some really interesting literature on BVP and https://boundaryvalueproblems.springeropen.com/articles/10.1186/s13661-024-01853-3


**Boundary layers in the ellipse test (after Wang \& Wang, 2024)**


**For the ideal ellipse**
 the coefficient of $\dfrac{\partial r}{\partial\theta}$ vanishes at
$$
\theta_{\min}=0,\quad
\theta_{\max}=\tfrac{\pi}{2},
$$
so the ends of the axes play the role of turning points in the sense of singular-perturbation theory.\footnote{A turning point is where the coefficient of the first derivative vanishes; cf.\ $\partial f/\partial r_\theta=0$ in Wang \& Wang.}
When $e\uparrow 1$ the minor-axis neighbourhood becomes a thin em boundary layer
of angular half-width
$$
\Delta\theta\sim\sqrt{1-e^{2}},
$$
inside which both $\partial\ln r/\partial\ln a$ and $\partial\ln r/\partial\ln b$
grow like $1/\!\sqrt{1-e^{2}}$.\\

**Uniform composite estimate**

Adapting the composite-expansion recipe of**wang2024multiple**,
patch the outer $O(1)$ radius with an inner stretched variable
$$
\eta=\frac{\theta-\theta_{\min}}{\sqrt{1-e^{2}}},
\qquad
e\lesssim 1.
$$
Keeping only the leading terms,
$$
r_{\text{comp}}(\eta)\;=\;
\underbrace{\frac{ab}{\sqrt{b^{2}+a^{2}\eta^{2}}}}_{\text{outer}}
\;+\;
\underbrace{A\exp\!\bigl[-m\eta\bigr]}_{\text{boundary layer}}
,
\quad
m=\frac{a}{b}\sqrt{1-e^{2}}.
$$
Matching at $a\to\infty$ fixes $A=0$, confirming that, for $e<0.9$, the layer merely sharpens the curvature without shifting the boundary by more than $\approx5$~ulp.

**Impact on classifier**
Insert $r_{\text{comp}}$ into the boundary test
$$
f(\mathbf x)=\frac{r_{px}}{r_{\text{comp}}(1+\delta)}\le1,
$$
and linearise as before.  The Wang \& Wang estimate
$\bigl|\delta r/r\bigr|_{\mathrm{layer}}\sim(1-e^{2})^{-1/2}\!,
$
gives
$$
\delta f
\;\approx\;
\varepsilon_r-\varepsilon_\delta-
\bigl(\varepsilon_a k_a+\varepsilon_b k_b\bigr)
\;+\;
O\!\left((1-e^{2})^{-\tfrac12}\varepsilon\right),
$$
so the extra boundary-layer term stays well below the jitter floor until $e\gtrsim0.97$
(semi-axis ratio $\gtrsim 5$).
Hence, under typical double precision, a rounding-induced class flip remains $\sim10^{12}$–$10^{14}$ times less probable than a biologically meaningful deformation.

**Essentially**
For $e\le0.9$ the original first-order error bounds already dominate;
beyond that, adopt the stretched‐variable layer width $\Delta\theta$
to keep perturbation estimates tight.


Following this are some practical guardrails:
'''

        EPS   = np.finfo(float).eps               # 2.22e-16
        scale = max(a, b)
        a_, b_ = a/scale, b/scale                 # 0 < a_,b_ ≤ 1
        den   = np.hypot(b_ * np.cos(theta),      # stable hypot again
                         a_ * np.sin(theta))
        r_ideal = (a_ * b_) / den * scale         # all intermediates ≤ 1

        rhs  = r_ideal * (1.0 + delta + EPS)      # one‑ulp safety band
        mask = r_px <= rhs
'''

Uses only standard NumPy; no performance hit. On CPUs with FMA, enable np.set_float32_matmul_precision('high') so the divisions use hardware FMA where possible.
Now I compare the use of float64 to float32

Bulletproofing my float32
'''

    a32 = np.float32(a)
    b32 = np.float32(b)
    scale = np.float32(max(a32, b32))
    a_ = a32/scale;  b_ = b32/scale           # keep ≤1 to dodge overflow
    eps32 = np.finfo(np.float32).eps          # 1.19e‑7
    den   = np.hypot(b_ * np.cos(theta),      # hypot is stable for float32 too
                     a_ * np.sin(theta))
    r_ideal32 = (a_ * b_) / den * scale
    mask = r_px32 <= r_ideal32 * (1 + delta32 + eps32)

'''

In [11]:
import imageio
import numpy as np
from skimage.draw import ellipse
import os
import cv2
import time
import math
print("Done Importing")

notebook_directory = os.getcwd()

print(f"Notebook directory: {notebook_directory}")

#was going to use float32 given that the final results are in uint8 and the former gives 7
#and float32 gives 7 decimal places of accuracy but the critical use to avoid cancellation error in  the following  c_coeff
"""
    VERY NUMERICALLY CRAZY BAD
        theta = np.deg2rad(angle_deg%360)
        cos_t = np.cos(theta)
        sin_t = np.sin(theta)

        a_coeff = (cos_t / semi_a) ** 2 + (sin_t / semi_b) ** 2
        b_coeff = (sin_t / semi_a) ** 2 + (cos_t / semi_b) ** 2
        c_coeff = 2 * sin_t * cos_t * (1/semi_a**2 - 1/semi_b**2)

"""

    #diff  = (b_hat - a_hat) * (b_hat + a_hat)
    #denom = (a_hat * a_hat) * (b_hat * b_hat)
    #delta  = diff / denom if abs(diff) > _DOUBLE_TINY else 0.0

# the product underflows for the diff and denom  underflow to float32 subnormal
# also reciprical- squares can exceed normal range
# rounding noise may accumulate past 7 decimal places
_DOUBLE_EPS  = np.finfo(np.float64).eps     # ≈ 2.22e‑16
_DOUBLE_TINY = np.finfo(np.float64).tiny    # ≈ 2.23e‑308



def ellipse_params_to_general_form(center_x: float,
                                   center_y: float,
                                   semi_a: float,
                                   semi_b: float,
                                   angle_deg: float,
                                   *,
                                   circle_rel_eps: float | None = None):
    """
    Parameters
              
    - center_x, center_y : float
        - Ellipse centre (h, k).
    - semi_a, semi_b : float
        - Semi‑major / semi‑minor axes *in pixels* (must be > 0).
    - angle_deg : float
        - Rotation angle theta in degrees.
    - circle_rel_eps : float or None, optional
        - Relative tolerance for deciding the ellipse is a circle.  If *None*
        (default), we use 2*machine ε(≈4.4e-16).  A value of, say,  1e‑8 duplicates the common graphics‑library threshold.
    """
    if semi_a <= 0 or semi_b <= 0:
        raise ValueError("semi_a and semi_b must be positive")


    semi_a   = np.float64(semi_a)
    semi_b   = np.float64(semi_b)
    center_x = np.float64(center_x)
    center_y = np.float64(center_y)

    #rel_diff threshold
    rel_eps = (2.0 * _DOUBLE_EPS) if circle_rel_eps is None else np.float64(circle_rel_eps)

    #circle shortcut – if |a - b| <= rel_eps * max(a, b), treat as circle.
    if abs(semi_a - semi_b) <= rel_eps * max(semi_a, semi_b):
        r      = np.float64(0.5) * (semi_a + semi_b)
        inv_r2 = np.float64(1.0) / (r * r)
        return {
            "a": inv_r2,  # A
            "b": inv_r2,  # C (equal for a circle)
            "c": np.float64(0.0),  # B
            "d": np.float64(1.0),
            "h": center_x,
            "w": center_y,
        }

    scale  = np.float64(max(semi_a, semi_b))
    a_hat, b_hat = semi_a / scale, semi_b / scale   #to bound 0 < a_hat, b_hat <= 1

    #b_hat or a_hat just means "normalized and i could be 1 or smaller"
    inv_a2 = np.float64(1.0) / (a_hat * a_hat)
    inv_b2 = np.float64(1.0) / (b_hat * b_hat)

    #c_coeff = 2 * sin_t * cos_t * (1/semi_a**2 - 1/semi_b**2)
    #init used this one but there is cancellation error in the parentheses so i made
    # delta, inv1 and inv2 for numerical stabilityinv_a2 = 1.0 / (a*a)
    #delta used later

    theta = np.deg2rad(angle_deg % 360)
    sin_t, cos_t = math.sin(theta), math.cos(theta)

    theta  = math.radians(angle_deg % 360.0)
    sin_t  = math.sin(theta)
    cos_t  = math.cos(theta)

    #                                                                   
    # 3Stable product for Δ = 1/a² − 1/b²  (avoids catastrophic cancellation)
    #     Δ = (b_hat − a_hat)(b_hat + a_hat) / (a_hat² b_hat²)
    #     Underflow safeguard: if Δ under‑flows to zero at float64, we can
    #     safely snap B to zero – numerically, the ellipse is essentially
    #     axis‑aligned.
    diff   = (b_hat - a_hat) * (b_hat + a_hat)
    denom  = (a_hat * a_hat) * (b_hat * b_hat)
    delta  = diff / denom if abs(diff) > _DOUBLE_TINY else 0.0
    1/semi_a**2 - 1/semi_b**2

    # coeffs - minimize rounding with fma
    sc = math.fma(sin_t, cos_t, 0.0) if hasattr(math, "fma") else sin_t * cos_t
    B  = 2.0 * sc * delta

    A  = (cos_t * cos_t) * inv_a2 + (sin_t * sin_t) * inv_b2
    C  = (sin_t * sin_t) * inv_a2 + (cos_t * cos_t) * inv_b2

    #undo rescaling - coeff for original pixel units
    scale2 = scale * scale
    A     /= scale2
    B     /= scale2
    C     /= scale2

    return {
        "a": A,
        "b": C,
        "c": B,
        "d": np.float64(1.0),
        "h": center_x,
        "w": center_y,
    }


Done Importing
Notebook directory: C:\Users\epw268\Documents\GitHub\yeastReverseImageSegmentation


In [4]:
def generate_uint8_labels(w: int, h: int, cells_data: dict,
                          *, use_vectorized: bool = True) -> np.ndarray:
    """
    Parameters:

    - w, h : int
        - Output image width and height **in pixels**.
    - cells_data : dict
        Expected structure::

            {
                "indices":      [1, 2, ...],              # uint8 IDs (1‑255)
                "fluorescence": [0.42, 0.17, ...],        # ignored here
                "shape":        [(a1, b1), (a2, b2), ...],
                "location":     [(x1, y1), (x2, y2), ...],
                "rotation":     [theta1, theta2, ...]             # degrees
            }

        - ``'indices'`` must contain unique integers <= 255.  All lists must be
        the same length.
    - use_vectorized : bool, default ``True``
        If ``True`` masks are generated with :func:`create_ellipse_mask_vectorized`.
        If ``False`` the pixel‑loop variant is used (slower, but easier to
        debug).


    Returns:
    - uint8_labels: np.ndarray of shape (h, w) with dtype=np.uint8
              -
    uint8_labels : np.ndarray, shape ``(h, w)``, dtype ``np.uint8``
        Background pixels hold 0; each ellipse interior is filled with its
        corresponding ID from ``cells_data['indices']``.

    - Notes
        - All heavy arithmetic is carried out in **float64** via
          :func:`ellipse_params_to_general_form`; conversion to ``uint8`` happens
          only at the final assignment step, ensuring numerical robustness.
        - Raises ``ValueError`` if a cell ID exceeds 255.

    """
    uint8_labels = np.zeros((h, w), dtype=np.uint8)
    #    indices,    shapes, locations,  rotations   = cells_data["indices"],  cells_data["shape"] ,   cells_data["location"], cells_data["rotation"]
    indices    = cells_data["indices"]
    shapes     = cells_data["shape"]       # list of (semi_a, semi_b)
    locations  = cells_data["location"]    # list of (x, y)
    rotations  = cells_data["rotation"]    # list of theta in degrees

    print(f"Generating uint8 labels for {len(indices)} cells...")
    print(f"Output array shape: {uint8_labels.shape}, dtype: {uint8_labels.dtype}")
    print(f"Using {'vectorized' if use_vectorized else 'mathematical'} ellipse generation")

    #mask generation method
    mask_fn = create_ellipse_mask_vectorized if use_vectorized else create_ellipse_mask_mathematical

    for cell_id, (semi_a, semi_b), (center_x, center_y), angle_rot_numbers in zip(indices, shapes, locations, rotations):
        if cell_id > 255:
            raise ValueError(f"Cell ID {cell_id} exceeds uint8 range 0‑255")

        coeffs   = ellipse_params_to_general_form(center_x, center_y, semi_a, semi_b, angle_rot_numbers)
        cell_mask = mask_fn(w, h, coeffs)
        uint8_labels[cell_mask] = cell_id

        pixel_count = np.sum(cell_mask)
        print(f"  Cell {cell_id}: center=({center_x},{center_y}), shape=({semi_a},{semi_b}), "
              f"rotation={angle_rot_numbers} number, pixels={pixel_count}")

    unique_labels = np.unique(uint8_labels)
    print(f"\nUint8 label summary:")
    print(f"  Unique values: {unique_labels}")
    print(f"  Background pixels (0): {np.sum(uint8_labels == 0)}")
    print(f"  Total labeled pixels: {np.sum(uint8_labels > 0)}")

    return uint8_labels

def _implicit_value(A: float, C: float, B: float, dx, dy):
    #compute A*dx² + C*dy² + B*dx*dy element‑wise (broadcast friendly)
    return A * dx * dx + C * dy * dy + B * dx * dy



Prototyping


In [16]:
if __name__ == "__main__":
    print("Running quick smoke tests for ellipse I/O + visualisation module…\n")

    # 1. Benchmark mathematical vs. vectorised
    compare_mask_generation_methods()

    # 2. Full pipeline on toy dataset
    labels, rgb = example_complete_pipeline()
    inspect_uint8_output(labels)
    print("Demo RGB image saved as demo_labels_vis.png\n")

    # 3. CV2 generator sanity check
    cv2_cells = {
        "indices": [1],
        "fluorescence": [0],
        "shape": [(25, 18)],
        "location": [(64, 64)],
        "rotation": [45],
    }
    cv2_labels = generate_uint8_labels_cv2(128, 128, cv2_cells)
    print("OpenCV generator produced label with unique values:", np.unique(cv2_labels))
    inspect_uint8_output(cv2_labels)
    print("All tests completed successfully.")

Running quick smoke tests for ellipse I/O + visualisation module…

=== Comparing Ellipse Mask Generation Methods ===
Image size: 128x128
Ellipse: center=(64,64), axes=(20,15), rotation=30°

Mathematical (pixel‑loop) method:
  Time: 6.72 ms | Pixels: 941

Vectorised method:
  Time: 6.72 ms | Pixels: 941
  Speedup: 1.0×

Masks identical: True
=== STEP 1: Generate uint8 labels ===

=== STEP 2: Save uint8 labels ===
Saved raw binary: yeast_segmentation.uint8
Saved NumPy array: yeast_segmentation.npy
Saved PNG: yeast_segmentation_labels.png
Saved metadata: yeast_segmentation_metadata.json

=== STEP 3: Visualise ===
Saved visualisation: yeast_segmentation_visual.png
=== Uint8 Label Array Inspection ===
Shape        : (128, 128)
Data type    : uint8
Memory       : 16384 bytes
Min | Max    : 0 | 9

Pixel counts per label:
  Background : 14279 px
  Cell 1     : 175 px
  Cell 2     : 279 px
  Cell 3     : 149 px
  Cell 4     : 347 px
  Cell 5     : 223 px
  Cell 6     : 253 px
  Cell 7     : 251